In [1]:
import pandas as pd
import numpy as np

In [2]:
encoding = 'ISO-8859-1'

In [3]:
train = pd.read_csv('/kaggle/input/hacke-earth-fibe/dataset/train.csv',encoding=encoding)
test = pd.read_csv('/kaggle/input/hacke-earth-fibe/dataset/test.csv',encoding=encoding)
ss = pd.read_csv('/kaggle/input/hacke-earth-fibe/dataset/sample_submission.csv',encoding=encoding)

In [4]:
train.head()

,text,target,Word Count
0,"python courses python courses, python exercise...",academic interests,125
1,the learning point open digital education. a r...,academic interests,147
2,"tech news, latest technology, mobiles, laptops...",academic interests,143
3,the best it certification materials in usa | k...,academic interests,364
4,"bioland scientific, for your research needs bi...",academic interests,176


In [8]:
train.shape

(697527, 3)

In [13]:
train['target'].value_counts()

target
academic interests                           58508
books and literature                         40435
healthy living                               30945
careers                                      30484
news and politics                            30197
shopping                                     29518
style and fashion                            29144
family and relationships                     28951
business and finance                         28038
automotives                                  28004
pharmaceuticals, conditions, and symptoms    26596
arts and culture                             26362
sports                                       24541
pets                                         24136
hobbies and interests                        23955
real estate                                  23845
food and drinks                              23434
home and garden                              23013
video gaming                                 22950
movies                  

In [5]:
pip install transformers torch scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [16]:
test

,text,Word Count,Index
0,"equl offers enzyme assay kits, reagent mixture...",353,Article_0
1,gauthmath: instant math questions solver for f...,112,Article_1
2,Whats the No. 1 cause of blindness in older ad...,340,Article_2
3,Surfers will ride a wave in the Amazon this we...,465,Article_3
4,"Why is the top of a leaf the most colorful, so...",269,Article_4
...,...,...,...
174377,his story will soon pale in comparison next to...,119,Article_174377
174378,Mundo.Take a look at a gallery of covers below...,115,Article_174378
174379,may change over the course of the game's devel...,141,Article_174379
174380,Mark Gustafson. If you've already watched it a...,101,Article_174380


# model

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train.head()

NameError: name 'train' is not defined

In [7]:
label_encoder = LabelEncoder()
train['category'] = label_encoder.fit_transform(train['target'])

In [8]:
df_sampled, _ = train_test_split(train, test_size=0.8, stratify=train['target'], random_state=42)

In [9]:
df_sampled.shape

(139505, 4)

In [10]:
df_sampled['target'].value_counts()

target
academic interests                           11702
books and literature                          8087
healthy living                                6189
careers                                       6097
news and politics                             6039
shopping                                      5904
style and fashion                             5829
family and relationships                      5790
business and finance                          5608
automotives                                   5601
pharmaceuticals, conditions, and symptoms     5319
arts and culture                              5272
sports                                        4908
pets                                          4827
hobbies and interests                         4791
real estate                                   4769
food and drinks                               4687
home and garden                               4603
video gaming                                  4590
movies                  

In [11]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_sampled['text'], df_sampled['category'], test_size=0.2, random_state=42)

In [12]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
train.columns

Index(['text', 'target', 'Word Count', 'category'], dtype='object')

In [22]:
train['Word Count'].describe()

count    697527.000000
mean        305.898281
std         137.798053
min         100.000000
25%         179.000000
50%         291.000000
75%         445.000000
max         500.000000
Name: Word Count, dtype: float64

In [25]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

# Tokenization (using batch processing for speed)
batch_size = 64  # Choose an appropriate batch size
train_encodings = []
test_encodings = []

for i in range(0, len(X_train), batch_size):
    batch = X_train[i:i + batch_size]
    encodings = tokenizer(batch, truncation=True, padding=True, max_length=300)
    train_encodings.append(encodings)

for i in range(0, len(X_test), batch_size):
    batch = X_test[i:i + batch_size]
    encodings = tokenizer(batch, truncation=True, padding=True, max_length=300)
    test_encodings.append(encodings)

# Combine the results into a single encoding
train_encodings = {k: [enc for d in train_encodings for enc in d[k]] for k in train_encodings[0].keys()}
test_encodings = {k: [enc for d in test_encodings for enc in d[k]] for k in test_encodings[0].keys()}


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [24]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

NameError: name 'AutoTokenizer' is not defined

In [13]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

In [14]:
# Tokeadd_suffixnization
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=300)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=300)


In [15]:
import torch

class ArticleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ArticleDataset(train_encodings, y_train.to_numpy())
test_dataset = ArticleDataset(test_encodings, y_test.to_numpy())


In [16]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=58,   # batch size per device during training
    per_device_eval_batch_size=58,    # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,3.265600
20,3.268500
30,3.256900
40,3.254100
50,3.238800
60,3.219900
70,3.187100
80,3.146500
90,3.081600
100,2.998200


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=3852, training_loss=0.7965007841525172, metrics={'train_runtime': 6610.4806, 'train_samples_per_second': 67.532, 'train_steps_per_second': 0.583, 'total_flos': 3.46645761394752e+16, 'train_loss': 0.7965007841525172, 'epoch': 4.0})

In [18]:
def predict_article_category(article):
    inputs = tokenizer(article, return_tensors='pt', truncation=True, padding=True, max_length=300)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class])[0]




In [19]:
def te(new_article):
    predicted_category = predict_article_category(new_article)
    return predicted_category

In [17]:
ss.head()

,target,Index
0,academic interests,Article_502
1,academic interests,Article_4578
2,academic interests,Article_4589
3,arts and culture,Article_11709
4,arts and culture,Article_12270


In [21]:
ss

,target,Index
0,academic interests,Article_502
1,academic interests,Article_4578
2,academic interests,Article_4589
3,arts and culture,Article_11709
4,arts and culture,Article_12270
5,books and literature,Article_17367
6,books and literature,Article_20404
7,business and finance,Article_25194
8,careers,Article_29227
9,food and drinks,Article_31560


In [18]:
test.head()

,text,Word Count,Index
0,"equl offers enzyme assay kits, reagent mixture...",353,Article_0
1,gauthmath: instant math questions solver for f...,112,Article_1
2,Whats the No. 1 cause of blindness in older ad...,340,Article_2
3,Surfers will ride a wave in the Amazon this we...,465,Article_3
4,"Why is the top of a leaf the most colorful, so...",269,Article_4


In [20]:
import torch

# Assuming you have already loaded your model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the appropriate device

def predict_article_category(article):
    inputs = tokenizer(article, return_tensors='pt', truncation=True, padding=True, max_length=290)
    
    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class = logits.argmax().item()
    return label_encoder.inverse_transform([predicted_class])[0]


In [ ]:
mm = test['text'].apply(te)

In [35]:
mm

0         academic interests
1         academic interests
2                     health
3          news and politics
4         academic interests
                 ...        
174377          video gaming
174378          video gaming
174379          video gaming
174380          video gaming
174381          video gaming
Name: text, Length: 174382, dtype: object

In [30]:
test

,text,Word Count,Index
0,"equl offers enzyme assay kits, reagent mixture...",353,Article_0
1,gauthmath: instant math questions solver for f...,112,Article_1
2,Whats the No. 1 cause of blindness in older ad...,340,Article_2
3,Surfers will ride a wave in the Amazon this we...,465,Article_3
4,"Why is the top of a leaf the most colorful, so...",269,Article_4
...,...,...,...
174377,his story will soon pale in comparison next to...,119,Article_174377
174378,Mundo.Take a look at a gallery of covers below...,115,Article_174378
174379,may change over the course of the game's devel...,141,Article_174379
174380,Mark Gustafson. If you've already watched it a...,101,Article_174380


In [31]:
ss['id']  = test['Index']

In [33]:
s = pd.read_csv('/kaggle/input/hacke-earth-fibe/dataset/sample_submission.csv')
s

,target,Index
0,academic interests,Article_502
1,academic interests,Article_4578
2,academic interests,Article_4589
3,arts and culture,Article_11709
4,arts and culture,Article_12270
5,books and literature,Article_17367
6,books and literature,Article_20404
7,business and finance,Article_25194
8,careers,Article_29227
9,food and drinks,Article_31560


In [37]:
test.head()

,text,Word Count,Index
0,"equl offers enzyme assay kits, reagent mixture...",353,Article_0
1,gauthmath: instant math questions solver for f...,112,Article_1
2,Whats the No. 1 cause of blindness in older ad...,340,Article_2
3,Surfers will ride a wave in the Amazon this we...,465,Article_3
4,"Why is the top of a leaf the most colorful, so...",269,Article_4


In [42]:
ss = pd.DataFrame()
ss['target'] = mm
ss['Index'] = test['Index']

In [44]:
ss.to_csv('sub_nlp_new.csv',index=False)

In [43]:
ss

,target,Index
0,academic interests,Article_0
1,academic interests,Article_1
2,health,Article_2
3,news and politics,Article_3
4,academic interests,Article_4
...,...,...
174377,video gaming,Article_174377
174378,video gaming,Article_174378
174379,video gaming,Article_174379
174380,video gaming,Article_174380
